As ET resolution is 1,8 visual angle and approximate smallest facial AOI area (nose in most cases) is 5.1 cm (11,25 cm^2) ⇒ maximum real world distance should be 162 cm. I can measure the pixel size of an AOI if i make some recordings where both people are not moving on that distance and calculate the pixel size of an AOI. Consequently filter AOIs which are smaller than that and not count them in the analysis.

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d

In [ ]:
# loading files from a recording of a face on the 162 cm distance and from the FaceMapper


In [ ]:
face_with_coordinates_df = pd.merge(
    fixations_df,
    face_df,
    on="start timestamp [ns]",
    how="inner"  # or "left" if you want to keep all fixations and NaNs for unmatched
)

# leave only fixations on face
face_with_coordinates_df = face_with_coordinates_df[face_with_coordinates_df['fixation on face'] == True]

#face_with_coordinates_df.head()
#print(len(face_with_coordinates_df))

### functions ###

# there are already AOI centers for eyes and nose provided by FaceMapper

## to find mouth centre point

def mouth_centre(m_left_x, m_left_y, m_right_x, m_right_y):
    # Calculate the center of the mouth by averaging the x and y coordinates
    mouth_x = (m_left_x + m_right_x) / 2
    mouth_y = (m_left_y + m_right_y) / 2
    return mouth_x, mouth_y


## to assign aois based on voronoi method for each row

def which_aoi(fix_x, fix_y, aoi_centers):
    
    """ Assigns the fixation to the nearest AOI center using Euclidean distance. """

    min_dist = float('inf')
    closest_aoi = None

    for aoi_id, (aoi_x, aoi_y) in aoi_centers.items():
        dist = np.sqrt((fix_x - aoi_x)**2 + (fix_y - aoi_y)**2)
        if dist < min_dist:
            min_dist = dist
            closest_aoi = aoi_id

    return closest_aoi


## to illustrate the results

def plot_fixation_and_aois(fix_x, fix_y, aoi_centers, assigned_aoi):
    fig, ax = plt.subplots(figsize=(5, 5))
    
    # Plot AOI centers
    for aoi_id, (aoi_x, aoi_y) in aoi_centers.items():
        ax.plot(aoi_x, aoi_y, 'o', label=aoi_id)
        ax.text(aoi_x + 5, aoi_y + 5, aoi_id, fontsize=9, color='black')

    # Plot fixation point
    ax.plot(fix_x, fix_y, 'rx', label='Fixation')
    ax.text(fix_x + 5, fix_y + 5, 'Fixation', color='red')

    # Draw a line from fixation to the assigned AOI
    if assigned_aoi in aoi_centers:
        aoi_x, aoi_y = aoi_centers[assigned_aoi]
        ax.plot([fix_x, aoi_x], [fix_y, aoi_y], 'r--', label='Assigned AOI link')

    ax.set_title('Voronoi-Based AOI Assignment')
    ax.set_xlabel('X [px]')
    ax.set_ylabel('Y [px]')
    ax.legend()
    ax.set_aspect('equal')
    plt.gca().invert_yaxis()  # Invert y-axis as working with screen coords
    plt.grid(True)
    plt.show()

In [ ]:
assigned_aois = []

for _, row in matched_df.iterrows():
    mouth_x, mouth_y = mouth_centre(
        row['mouth left x [px]'], row['mouth left y [px]'],
        row['mouth right x [px]'], row['mouth right y [px]']
    )
    
    aoi_centers = {
        'left_eye': (row['eye left x [px]'], row['eye left y [px]']),
        'right_eye': (row['eye right x [px]'], row['eye right y [px]']),
        'nose': (row['nose x [px]'], row['nose y [px]']),
        'mouth': (mouth_x, mouth_y)
    }

    assigned_aoi = which_aoi(row['fixation x [px]'], row['fixation y [px]'], aoi_centers)

    #plot_fixation_and_aois(row['fixation x [px]'], row['fixation y [px]'], aoi_centers, assigned_aoi)

    assigned_aois.append(assigned_aoi)

matched_df['assigned_aoi'] = assigned_aois# Select the first row


row = matched_df.iloc[27]

# Calculate mouth center
mouth_x, mouth_y = mouth_centre(
    row['mouth left x [px]'], row['mouth left y [px]'],
    row['mouth right x [px]'], row['mouth right y [px]']
)

# Define AOI centers with colors
aoi_centers = {
    'left_eye': ((row['eye left x [px]'], row['eye left y [px]']), 'blue'),
    'right_eye': ((row['eye right x [px]'], row['eye right y [px]']), 'green'),
    'nose': ((row['nose x [px]'], row['nose y [px]']), 'orange'),
    'mouth': ((mouth_x, mouth_y), 'purple')
}

# Convert AOI coords to array for Voronoi
points = np.array([coords for coords, _ in aoi_centers.values()])

# Create Voronoi diagram
vor = Voronoi(points)

# Plot
fig, ax = plt.subplots(figsize=(6, 6))
voronoi_plot_2d(vor, ax=ax, show_vertices=False, line_colors='gray', line_width=1.5, line_alpha=0.6, point_size=0)

# Plot AOI points with different colors
for label, ((x, y), color) in aoi_centers.items():
    ax.plot(x, y, 'o', color=color, label=label)

# Plot fixation point
fix_x = row['fixation x [px]']
fix_y = row['fixation y [px]']
ax.plot(fix_x, fix_y, 'rx', markersize=12, label='Fixation')

# Draw bounding box from p1 (top-left) to p2 (bottom-right)
x1 = row['p1 x [px]']
y1 = row['p1 y [px]']
x2 = row['p2 x [px]']
y2 = row['p2 y [px]']
bbox_w = x2 - x1
bbox_h = y2 - y1
rect = plt.Rectangle((x1, y1), bbox_w, bbox_h, linewidth=2, edgecolor='black', facecolor='none', label='Bounding Box')
ax.add_patch(rect)

# Add legend outside plot
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Set title and display
ax.set_title(f"AOI assighment visualization ({assigned_aois[27]})")
plt.gca().invert_yaxis()
plt.axis('equal')
#plt.tight_layout()
plt.show()
